In [ ]:
type Card
    rank::Int64
    suit::Char
end

type Deck
    cards::Array{Card}
end

type Hand
    cards::Array{Card}
end


In [33]:
function newDeck()
    deck = Array{Card}(0)
    for r = 2:14
        for s = ['s','h','d','c']
            push!(deck,Card(r,s))
        end
    end
    for i=1:52
        j = rand(1:52)
        deck[i], deck[j] = deck[j], deck[i]
    end
    return deck
end

function rankString(val)
    if val < 11
        return string(val)
    elseif val == 11
        return "Jack"
    elseif val == 12
        return "Queen"
    elseif val == 13
        return "King"
    else
        return "Ace"
    end
end

function emptyHand()
    return Array{Card}(0)
end

function draw!(deck,hand)
    return push!(hand,pop!(deck))
end

draw! (generic function with 1 method)

In [57]:
function evaluateHand(hand)
    flush = false
    flushSuit = nothing
    suits = Dict('s' => 0, 'h' => 0, 'd' => 0, 'c' => 0)
    valArr = []
    for i=1:size(hand,1)
        suits[hand[i].suit] += 1
        push!(valArr, hand[i].rank)
    end
    for suit in keys(suits)
        if suits[suit] >= 5
            flush = true
            flushSuit = suit
            # now check if it is straight flush
            vals = Dict{Int64,Int64}()
            for i=1:size(hand,1)
                card = hand[i]
                if card.suit == flushSuit
                    vals[card.rank] = 1
                end    
            end
            consec = 0
            vals[1] = get(vals, 14, 0) # since ace can be high or low
            for i=14:-1:1
                if get(vals, i, 0) > 0
                    consec += 1
                else
                    consec = 0
                end
                if consec == 5
                    high = i+4
                    text = "Straight Flush, " * rankString(high) * " high"
                    score = (9,high)
                    return (text,score)
                end
            end
        end
    end
    vals = Dict{Int64,Int64}()
    for i=1:size(hand,1)
        card = hand[i]
        vals[card.rank] = get(vals,card.rank,0) + 1
    end
    threes = Array{Int64}(0)
    twos = Array{Int64}(0)
    for key in keys(vals)
        if vals[key] == 4
            ## 4 of a kind
            high = maximum(filter(e->e != key, valArr))
            text = "Four of a Kind, " * rankString(key) * " with " * rankString(high) * " high"
            score = (8, key * 14 + high)
            return (text, score)
        elseif vals[key] == 3
            push!(threes, key)
            push!(twos, key)
        elseif vals[key] == 2
            push!(twos, key)
        end
    end
    if size(threes,1) > 0 && size(twos,1) > 1
        three = maximum(threes)
        two = maximum(filter(e->e != three, twos))
        text = "Full House, " * rankString(three) * " full of " * rankString(two)
        score = (7, three * 14 + two)
        return (text, score)
    end
    if flush
        high = 0
        for i=1:size(hand,1)
            card = hand[i]
            if card.suit == flushSuit
                high = max(high, card.rank)
            end
        end
        text = "Flush, " * rankString(high) * " high"
        score = (6, high)
        return (text, score)
    end
    # check for straight
    consec = 0
    vals[1] = get(vals, 14, 0) # since ace can be high or low
    for i=14:-1:1
        if get(vals, i, 0) > 0
            consec += 1
        else
            consec = 0
        end
        if consec == 5
            high = i+4
            text = "Straight, " * rankString(high) * " high"
            score = (5, high)
            return (text, score)
        end
    end
    if size(threes,1) > 0
        three = threes[1]
        high1 = maximum(filter(e->e != three, valArr))
        high2 = maximum(filter(e->!(e in [three,high1]), valArr))
        text = "Three of a kind, "*rankString(three)*" highs: "*rankString(high1)*","*rankString(high2)
        score = (4, three * 14^2 + high1*14 + high2)
        return (text, score)
    end
    if size(twos,1) > 1
        highPair = maximum(twos)
        lowPair = maximum(filter(e->e != highPair, twos))
        high = maximum(filter(e->!(e in [highPair,lowPair]), valArr))
        text = "Two pair, "*rankString(highPair)*" and "*rankString(lowPair)*", "*rankString(high)*" high"
        score = (3, highPair*14^2 + lowPair*14 + high)
        return (text, score)
    end
    if size(twos,1) == 1
        pair = maximum(twos)
        high1 = maximum(filter(e->e != pair, valArr))
        high2 = maximum(filter(e->!(e in [pair,high1]), valArr))
        high3 = maximum(filter(e->!(e in [pair,high1,high2]), valArr))
        text = "Pair: "*rankString(pair)*" highs: "*rankString(high1)*","*rankString(high2)*","*rankString(high3)
        score = (2, pair*14^3 + high1*14^2 + high2*14 + high3)
        return (text, score)
    end
    # no hand
    high1 = maximum(valArr)
    high2 = maximum(filter(e->e != high1, valArr))
    high3 = maximum(filter(e->!(e in [high1,high2]), valArr))
    high4 = maximum(filter(e->!(e in [high1,high2,high3]), valArr))
    high5 = maximum(filter(e->!(e in [high1,high2,high3,high4]), valArr))
    text = "Highs: "*rankString(high1)*","*rankString(high2)*","*rankString(high3)*","*rankString(high4)*","*rankString(high5)
    score = (1, high1*14^4 + high2*14^3 + high3*14^2 + high4*14 + high5)
    return (text, score)
end



evaluateHand (generic function with 1 method)

In [ ]:
for j=1:1000000
    deck = newDeck()
    hand = emptyHand()
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    draw!(deck,hand)
    evaluateHand(hand)
    #println(hand)
end

In [23]:
evaluateHand(hand)

Asdfasdfasdfa


("Straight Flush, Jack High",(9,11))

In [62]:
hand = [Card(10,'c'), Card(4,'c'), Card(5,'c'), Card(6,'c'), Card(14,'s'),Card(2,'h'),Card(12,'s')]

7-element Array{Card,1}:
 Card(10,'c')
 Card(4,'c') 
 Card(5,'c') 
 Card(6,'c') 
 Card(14,'s')
 Card(2,'h') 
 Card(12,'s')

In [63]:
println(evaluateHand(hand))

("Highs: Ace,Queen,10,6,5",(1,572801))


In [11]:
typeof(a)

Tuple{Int64,Int64}